In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.notebook import tqdm
from ast import literal_eval
import os
import sys
import pickle 

tqdm.pandas()

# Load Data

In [ ]:
review1 = pd.read_csv("review_data_first.csv")
review2 = pd.read_csv("review_data_second.csv")
review3 = pd.read_csv("review_data_third.csv")
review4 = pd.read_csv("review_data_fourth.csv")
review5 = pd.read_csv("review_data_fifth.csv")
review6 = pd.read_csv("review_data_sixth.csv")

In [ ]:
review_final = pd.concat([review1,review2,review3,review4,review5,review6]).reset_index(drop=True)
review_final = review_final.drop_duplicates(keep="first").reset_index(drop=True)

# 리뷰 데이터 전처리 전 EDA

In [ ]:
review_final.head(10)

In [ ]:
review_final.info()

In [ ]:
for i in range(1,6):
    print("rated %d : %.2f %% (%d)" %(i, len(review_final[review_final["rating"]==i])/len(review_final) * 100,
                                     len(review_final[review_final["rating"]==i])))
print("total ratings : %d" %len(review_final))

plt.xticks(range(1,10))
sns.histplot(review_final["rating"])
plt.figure()

In [ ]:
pd.set_option("max_rows",None)
review_final.loc[100000:100100,"review_text"]

In [ ]:
# Preprocess columns except review text
review = review_final.copy()
review["name"]=review_final["category"].str.split("\n").apply(lambda x : x[-1])
review = review.drop(["category", "price"], axis=1)
review

# Data Cleaning

# Separate Sentences

In [ ]:
!pip install kss

In [ ]:
import re
import kss
import string

In [ ]:
review["review_text"] = review["review_text"].apply(lambda x : kss.split_sentences(x))

## Clean Data

In [ ]:
# 각 문장 Data Cleaning (URL, HTML, emoji, punctuation)
def clean_text(texts):
  cleaned_text = []
  for text in texts:
    # remove URL
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'',text)
    # remove HTML
    html=re.compile(r'<.*?>')
    text = html.sub(r'',text)
    # remove emoji
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # remove punctuation
    table = str.maketrans('','',string.punctuation)
    text = text.translate(table)
    # remove special tokens
    special_token = re.compile(r'[^\w]')
    text = special_token.sub(r' ', text)
    cleaned_text.append(text)
  return cleaned_text

In [ ]:
# Apply data cleaning
review["review_text"] = review["review_text"].apply(lambda x : clean_text(x))

## 띄어쓰기

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from pykospacing import Spacing
# Text 모든 공백 제거
review["review_text"] = review["review_text"].str.replace(" ","")

# 빠른 처리를 위해 문장 합치기 "."으로 나중에 구분
review["review_text"] =  review["review_text"].progress_apply(lambda x : ".".join(x))

# null 데이터 삭제
review = review.dropna().reset_index(drop=True)

In [ ]:
# 띄어쓰기 적용
spacing = Spacing()
review["review_text"] = review["review_text"].progress_apply(lambda x : spacing(x))
review["review_text"][:10]

## 맞춤법 교정

In [ ]:
!pip install requests

In [ ]:
from hanspell import spell_checker

In [ ]:
def spell_check(x):
    try:
        return spell_checker.check(x["review_text"]).checked
    except:
        print("error occured at {}".format(x.name))
        time.sleep(60)
        return "error"

In [ ]:
review[25:29].apply(lambda x : spell_check(x), axis=1)

In [ ]:
# 3만개씩 나누어서 진행 -> Disconnetction error 발생
for i in range(len(review) // 30000 + 1):
    if i == 9:
        review[30000*i:].to_csv("review_cleaned_{}.csv".format(i), index=False)
    else:
        review[30000*i:30000*(i+1)].to_csv("review_cleaned_{}.csv".format(i), index=False)

In [ ]:
# for i in range(5, len(review) // 30000 + 1):
#     review_section = pd.read_csv("review_cleaned_{}.csv".format(i))
#     review_section["review_text"] = review_section.progress_apply(lambda x: spell_check(x), axis=1)
#     review_section.to_csv("review_cleaned_finished_{}.csv".format(i), index=False)
#     print("{}th process succeed".format(i))
#     time.sleep(60)

In [ ]:
# 맞춤법 확인한 데이터 합치기
review_spell_checked = pd.read_csv("review_cleaned_finished_0.csv")
for i in range(1, len(review) // 30000 + 1):
    review_temp = pd.read_csv("review_cleaned_finished_{}.csv".format(i))
    review_spell_checked = pd.concat([review_spell_checked, review_temp]).reset_index(drop=True)

In [ ]:
# Null 값 error로 변경
review_spell_checked["review_text"] = review_spell_checked["review_text"].fillna("error")

# Error 발생한 데이터 다시 맞춤법 검사
error_index = review_spell_checked[review_spell_checked["review_text"] == "error"].index
review_spell_checked.loc[error_index, "review_text"] = review.loc[error_index,:].progress_apply(lambda x: spell_check(x), axis=1)

In [ ]:
review_spell_checked[review_spell_checked["review_text"] == "error"]

In [ ]:
# 각 문장 Data Cleaning (URL, HTML, emoji, punctuation)
import re
import string

def clean_text(text):
    # remove URL
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'',text)
    # remove HTML
    html=re.compile(r'<.*?>')
    text = html.sub(r'',text)
    # remove emoji
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # remove punctuation
    table = str.maketrans('','',string.punctuation)
    text = text.translate(table)
    # remove special tokens
    special_token = re.compile(r'[^\w]')
    text = special_token.sub(r' ', text)
    return text.strip()

In [ ]:
# 맞춤법 검사 Error 발생한 데이터 삭제
review_spell_checked = review_spell_checked.drop(review_spell_checked[review_spell_checked["review_text"] == "error"].index, axis=0).reset_index(drop=True)
# 특수 문자 제거
review_spell_checked["review_text"] = review_spell_checked["review_text"].progress_apply(lambda x: clean_text(x))

## Normalize

repeat_normalize : 반복되는 것 정제
emoticon_normalize : 자음/모음의 이모티콘이 만나는 점에서 완전 한글이 만들어지는 경우 정제

In [ ]:
!pip install soynlp

In [ ]:
from soynlp.normalizer import *
emoticon_normalize('할머니 댁에 하나 집에 하나시켰어요. 덕분에 이쁜 손녀 이쁜 딸 됬네요 ㅎㅎ.', num_repeats=1)

In [ ]:
review_spell_checked["review_text"] = review_spell_checked["review_text"].progress_apply(lambda x : repeat_normalize(x, num_repeats=2))
review_spell_checked["review_text"] = review_spell_checked["review_text"].progress_apply(lambda x : emoticon_normalize(x, num_repeats=2))

In [2]:
# 전처리 완료한 데이터 저장
review_spell_checked.to_csv("review_spell_checked.csv", index=False)

NameError: name 'review_spell_checked' is not defined

## 형태소 분석 및 품사 태깅(Pos Tag)

### 1.형태소 분석을 기반으로 한 명사 추출 

### 2.구문분석을 기반으로 관형사의 수식을 받는 명사 추출

### 3. 구문분석을 기반으로 부사의 수식을 받는 단어 추출 

## Koalanlp 활용

In [3]:
# 전처리 완료한 데이터 불러오기
review_spell_checked = pd.read_csv("review_spell_checked.csv").dropna().reset_index(drop=True)

In [4]:
from koalanlp.Util import initialize, finalize

# 꼬꼬마와 ETRI 분석기의 2.0.4 버전을 참조합니다.
initialize(java_options="-Xmx4g", EUNJEON="latest", KMR="latest", KKMA="latest", HNN="latest")

[koalanlp.jip] [INFO] Latest version of kr.bydelta:koalanlp-eunjeon (2.1.6) will be used.
[koalanlp.jip] [INFO] Latest version of kr.bydelta:koalanlp-kmr (2.1.4) will be used.
[koalanlp.jip] [INFO] Latest version of kr.bydelta:koalanlp-kkma (2.1.4) will be used.
[koalanlp.jip] [INFO] Latest version of kr.bydelta:koalanlp-hnn (2.1.4) will be used.
[root] Java gateway started with port number 35289
[root] Callback server will use port number 25334
[koalanlp.jip] JVM initialization procedure is completed.


In [5]:
from koalanlp import API
from koalanlp.proc import Parser
from koalanlp.proc import Tagger
from koalanlp.types import PhraseTag

In [6]:
# review : each review row as an input
review_df = review_spell_checked.copy() # before starting a function copy the data
noun_block_dict = {} # key : word, value : [total_freq, mod_freq]
aspect_block_dict = {} # key : word, value : [total_freq]
key_tags =  ["NNG", "NNP","VV", "VA", "XR"] # only consider these tags
mecab_tagger = Tagger(API.EUNJEON)
kkma_parser = Parser(API.KKMA)

print("noun_block_dict :", noun_block_dict)
print("aspect_block_dict :", aspect_block_dict)

noun_block_dict : {}
aspect_block_dict : {}


In [7]:
sample_sentence = review_spell_checked.loc[1000, "review_text"]
print(kkma_parser(mecab_tagger(sample_sentence)))
print(mecab_tagger(sample_sentence)[0].getNouns())

[역시 나 만족하는 막걸리]
[나 = 나/NP, 막걸리 = 막걸리/NNG]


In [8]:
print("noun_block_dict :",noun_block_dict)
print("aspect_block_dict :",aspect_block_dict)
review_df

noun_block_dict : {}
aspect_block_dict : {}


,name,rating,review_text
0,복순도가,5.0,한 달 사용기 부모님과 한잔하면서 좋은 시간 보냈어요 부모님 댁으로 바로 배송했더니...
1,복순도가,5.0,막걸리 좋아하시는 어른께 명절 선물로 보내드렸습니다 명절에다 모여서 드시라고 3병 ...
2,복순도가,5.0,할머니 댁에 하나 집에 하나 시켰어요 덕분에 이쁜 손녀 이쁜 딸 됐네요 ㅎㅎ 개인적...
3,복순도가,5.0,설날과 집안 행사를 핑계로 구매했습니다 병 딸 때 조심해야 하는 건 원래 알고 있어...
4,복순도가,5.0,제가 술은 따로 즐겨 먹진 않는데 이상하게 막걸리는 한두 잔 정도 마십니다 ㅎㅎ론 ...
...,...,...,...
279406,맑은내일,5.0,배송도 빨랐고 맛도 있어요
279407,맑은내일,5.0,귀여워요 미니 미니 한 게
279408,맑은내일,5.0,선물용으로 샀는데 만족합니다
279409,맑은내일,4.0,달지 않고 부드러워 마시기 적당했어요


In [9]:
def get_aspects(review, noun_dict, aspect_dict, key_tags, tagger, parser):
    try:
        # for each row
        candidate_dictionary = {} # for saving ids for candidates
        tagged_review = tagger(review)
        parsed_review = parser(tagged_review)
        nouns = parsed_review[0].getNouns() # extract 어절 used as noun
        modifiers = parsed_review[0].getModifiers() # extract 어절 used as modifiers
        # extract key morphs from each noun phrase (1), (2)
        for noun in nouns:
            extracted_noun = ""
            for morph in noun.morphemes:
                # extract just key_tags
                if morph.tag in key_tags:
                    extracted_noun += morph.getSurface()
                if extracted_noun != "": # if it is not redundant morph
                    # Save at candidate dictionary
                    if noun.id not in candidate_dictionary.get(extracted_noun , []):
                        candidate_dictionary[extracted_noun] = candidate_dictionary.get(extracted_noun , []) + [noun.id]
                    # save at noun_dict
                    lock.acquire() # Prevent shared memory issue
                    noun_dict[extracted_noun] = noun_dict.get(extracted_noun, [0, 0])
                    noun_dict[extracted_noun][0] += 1
                    lock.release() # Make sure to realese lock
                    # Extract nouns that is modified
                    noun_edge = noun.getDependentEdges()
                    if (noun_edge != []) and (noun_edge[0].label == "MOD"):
                        # save at noun_dict adj_freq
                        lock.acquire() # Prevent shared memory issue
                        noun_dict[extracted_noun] = noun_dict.get(extracted_noun, [0, 0])
                        noun_dict[extracted_noun][1] += 1
                        lock.release() # Make sure to realese lock
        # extract key morphs from each modifier (3)
        for modifier in modifiers:
            # get modified word
            modified_word = modifier.getGovernorEdge().src
            extracted_mod = ""
            if modified_word != None:
                for morph in modified_word.morphemes:
                    # extract just key_tags
                    if morph.tag in key_tags:
                        extracted_mod += morph.getSurface()
                    if extracted_mod != "": # if it is not redundant morph
                        # Save at candidate dictionary
                        if modified_word.id not in candidate_dictionary.get(extracted_mod , []):
                            candidate_dictionary[extracted_mod] = candidate_dictionary.get(extracted_mod , []) + [modified_word.id]
                        # save at aspect_dict
                        lock.acquire() # Prevent shared memory issue
                        aspect_dict[extracted_mod] = aspect_dict.get(extracted_mod, 0) + 1
                        lock.release() # Make sure to realese lock
        return candidate_dictionary
    except:
        print("error occured")
        return "error"

In [10]:
# Multiprocessing
from threading import Lock
import swifter
import multiprocessing

lock = multiprocessing.Lock()
swifter.set_defaults(npartitions=8)

In [ ]:
# apply single core
# Time: 7 mins 4 secs for 1000 rows / 42 mins for 80000 rows

# review_df.loc[:10000,"candiate_ids"] = review_df.loc[:10000, "review_text"].progress_apply(lambda x :get_aspects(x, noun_block_dict, aspect_block_dict,
                                                                       key_tags, mecab_tagger, kkma_parser))

In [ ]:
# apply function by multiprocessing (4 cores)
# default (max usage)= cpu_counts x 2
# Time: 4 mins 55 secs for 1000 rows / 52 mins for 80000 rows

# review_df.loc[:10000,"candiate_ids"] = review_df.loc[:10000, "review_text"].swifter.apply(lambda x :get_aspects(x, noun_block_dict, aspect_block_dict,
                                                                                                                              key_tags, mecab_tagger, kkma_parser))

In [29]:
batch_size = 10000
for i in range(17, (len(review_df) // batch_size)+1):
    print("{}th process start".format(i))
    if i == (len(review_df) // batch_size):
        review_df.loc[batch_size*i:, "candiate_ids"] = review_df.loc[batch_size*i:, "review_text"].progress_apply(lambda x :get_aspects(x, noun_block_dict, aspect_block_dict,
                                                                                                                           key_tags, mecab_tagger, kkma_parser))
        
    else:
        review_df.loc[batch_size*i:batch_size*(i+1), "candiate_ids"] = review_df.loc[batch_size*i:batch_size*(i+1), "review_text"].progress_apply(lambda x :get_aspects(x, noun_block_dict, aspect_block_dict,
                                                                                                                           key_tags, mecab_tagger, kkma_parser))
    review_df.to_csv("main_dictionary.csv", index=False)
    with open('noun_dictionary.pkl', 'wb') as f:
        pickle.dump(noun_block_dict, f)

    with open('aspect_dictionary.pkl', 'wb') as f:
        pickle.dump(aspect_block_dict, f)
    
    print("{}th batch succeed".format(i))

17th process start


[root] Java에서 처리하던 중에 문제가 발생했습니다. 문제가 계속된다면, Issue를 등록해주세요.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/koalanlp/proc.py", line 237, in analyze
    result.append(Sentence.fromJava(self.__api.analyze(ref)))
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.8/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling o210.analyze.
: java.lang.IllegalStateException: There are more than two properties belong to the same key: DEP_GOVERNOR
	at kr.bydelta.koala.data.CanHaveProperty.setProperty$koalanlp_core(data.kt:116)
	at kr.bydelta.koala.data.DepEdge.<init>(data.kt:680)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:283)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:201)
	at kr.bydelta.koala.proc.CanAnalyzeProperty$DefaultImpls.analyze(

error occured


[root] Java에서 처리하던 중에 문제가 발생했습니다. 문제가 계속된다면, Issue를 등록해주세요.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/koalanlp/proc.py", line 237, in analyze
    result.append(Sentence.fromJava(self.__api.analyze(ref)))
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.8/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling o210.analyze.
: java.lang.IllegalStateException: There are more than two properties belong to the same key: DEP_GOVERNOR
	at kr.bydelta.koala.data.CanHaveProperty.setProperty$koalanlp_core(data.kt:116)
	at kr.bydelta.koala.data.DepEdge.<init>(data.kt:680)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:283)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:201)
	at kr.bydelta.koala.proc.CanAnalyzeProperty$DefaultImpls.analyze(

error occured


...

error occured

26th batch succeed
27th process start


[root] Java에서 처리하던 중에 문제가 발생했습니다. 문제가 계속된다면, Issue를 등록해주세요.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/koalanlp/proc.py", line 237, in analyze
    result.append(Sentence.fromJava(self.__api.analyze(ref)))
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.8/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling o210.analyze.
: java.lang.IllegalStateException: There are more than two properties belong to the same key: DEP_GOVERNOR
	at kr.bydelta.koala.data.CanHaveProperty.setProperty$koalanlp_core(data.kt:116)
	at kr.bydelta.koala.data.DepEdge.<init>(data.kt:680)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:283)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:201)
	at kr.bydelta.koala.proc.CanAnalyzeProperty$DefaultImpls.analyze(

error occured


[root] Java에서 처리하던 중에 문제가 발생했습니다. 문제가 계속된다면, Issue를 등록해주세요.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/koalanlp/proc.py", line 237, in analyze
    result.append(Sentence.fromJava(self.__api.analyze(ref)))
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.8/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling o210.analyze.
: java.lang.IllegalStateException: There are more than two properties belong to the same key: DEP_GOVERNOR
	at kr.bydelta.koala.data.CanHaveProperty.setProperty$koalanlp_core(data.kt:116)
	at kr.bydelta.koala.data.DepEdge.<init>(data.kt:680)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:283)
	at kr.bydelta.koala.kkma.Parser.attachProperty(proc.kt:201)
	at kr.bydelta.koala.proc.CanAnalyzeProperty$DefaultImpls.analyze(

error occured

27th batch succeed


In [34]:
print("noun_block_dict length :", len(noun_block_dict))
print("aspect_block_dict length :", len(aspect_block_dict))

noun_block_dict length : 16190
aspect_block_dict length : 7252


In [35]:
# Load saved noun_block_dict, aspect_block_dict
# noun_block_dict : 12000 at 170000 iterations / 16190 
# aspect_block_dict :5000 at 170000 iterations / 7252
with open('noun_dictionary.pkl', 'rb') as f:
    noun_block_dict = pickle.load(f)
    
with open('aspect_dictionary.pkl', 'rb') as f:
    aspect_block_dict = pickle.load(f)

In [19]:
# Load saved dataframe
review_df = pd.read_csv("main_dictionary.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


169990                                          {'맛있': [1]}
169991                           {'선물': [0, 4], '패키지': [2]}
169992          {'사람': [2], '반응': [3], '기대': [9], '좋': [5]}
169993                    {'쇼핑': [2], '쇼핑백': [2], '없': [4]}
169994    {'휴가': [1], '후': [2], '직원': [3], '선물': [4], '구...
169995                                          {'선물': [0]}
169996                                {'목': [1], '넘기': [2]}
169997            {'맛': [1], '포장': [3], '있': [2], '좋': [9]}
169998          {'사용': [2], '기': [3], '선물': [4], '만족': [7]}
169999                                          {'만족': [1]}
170000    {'술': [0], '유자': [7], '향': [8], '얼': [11], '얼음...
170001                                                  NaN
170002                                                  NaN
170003                                                  NaN
170004                                                  NaN
170005                                                  NaN
170006                                  

In [65]:
temp = dict(filter(lambda elem:len(elem[0])>1 or elem[0]=="맛", noun_block_dict.items()))
list(dict(sorted(temp.items(), key=lambda item: item[1], reverse=True)).keys())[:100]

['맛',
 '선물',
 '막걸리',
 '배송',
 '사용',
 '구매',
 '포장',
 '넘기',
 '탄산',
 '최고',
 '와인',
 '다음',
 '친구',
 '생각',
 '느낌',
 '가격',
 '부모',
 '지인',
 '가족',
 '단맛',
 '제품',
 '이번',
 '사람',
 '마시',
 '소주',
 '주문',
 '명절',
 '정도',
 '처음',
 '입맛',
 '박스',
 '복분자',
 '상품',
 '도수',
 '개인',
 '수도',
 '만족',
 '아버지',
 '마음',
 '예정',
 '기분',
 '복분',
 '기대',
 '상태',
 '추천',
 '숙취',
 '고급',
 '추석',
 '전통주',
 '어른',
 '마트',
 '남편',
 '알코올',
 '아빠',
 '택배',
 '전통',
 '예전',
 '스파클',
 '술맛',
 '음식',
 '시간',
 '패키지',
 '인터넷',
 '리뷰',
 '마을',
 '후기',
 '일반',
 '사진',
 '신맛',
 '안주',
 '취향',
 '신랑',
 '엄마',
 '냉장고',
 '여자',
 '맛있',
 '특유',
 '샴페인',
 '디자인',
 '안동소주',
 '모임',
 '이상',
 '얼음',
 '강추',
 '소곡주',
 '유자',
 '음료',
 '냄새',
 '호불호',
 '주변',
 '맥주',
 '시중',
 '복수',
 '어르신',
 '부드럽',
 '덕분',
 '하루',
 '탁주',
 '안동',
 '부담']

In [62]:
temp2 = dict(filter(lambda elem:len(elem[0])>1 or elem[0]=="맛", aspect_block_dict.items()))
list(dict(sorted(temp.items(), key=lambda item: item[1], reverse=True)).keys())[:100]

['맛',
 '선물',
 '막걸리',
 '배송',
 '사용',
 '구매',
 '포장',
 '넘기',
 '탄산',
 '최고',
 '와인',
 '다음',
 '친구',
 '생각',
 '느낌',
 '가격',
 '부모',
 '지인',
 '가족',
 '단맛',
 '제품',
 '이번',
 '사람',
 '마시',
 '소주',
 '주문',
 '명절',
 '정도',
 '처음',
 '입맛',
 '박스',
 '복분자',
 '상품',
 '도수',
 '개인',
 '수도',
 '만족',
 '아버지',
 '마음',
 '예정',
 '기분',
 '복분',
 '기대',
 '상태',
 '추천',
 '숙취',
 '고급',
 '추석',
 '전통주',
 '어른',
 '마트',
 '남편',
 '알코올',
 '아빠',
 '택배',
 '전통',
 '예전',
 '스파클',
 '술맛',
 '음식',
 '시간',
 '패키지',
 '인터넷',
 '리뷰',
 '마을',
 '후기',
 '일반',
 '사진',
 '신맛',
 '안주',
 '취향',
 '신랑',
 '엄마',
 '냉장고',
 '여자',
 '맛있',
 '특유',
 '샴페인',
 '디자인',
 '안동소주',
 '모임',
 '이상',
 '얼음',
 '강추',
 '소곡주',
 '유자',
 '음료',
 '냄새',
 '호불호',
 '주변',
 '맥주',
 '시중',
 '복수',
 '어르신',
 '부드럽',
 '덕분',
 '하루',
 '탁주',
 '안동',
 '부담']

In [ ]:
# 사용이 끝나면 아래와 같이 사용을 종료합니다.
finalize()

# 전처리 데이터 저장

In [ ]:
# Save preprocessed data
review.to_csv("review_cleaned.csv", index=False)

In [ ]:
# 저장된 csv 불러오가
review = pd.read_csv("review_cleaned.csv")
# 리스트로 element 저장시 사용해야함
# review["review_text"] = review["review_text"].apply(lambda x : literal_eval(x))
review = review.dropna().reset_index(drop=True)
review